In [ ]:
!pip install hazm

In [1]:
import numpy as np
import pandas as pd
import hazm

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import BertTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv', on_bad_lines='skip' , delimiter='\t')
data.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [3]:
data.dropna(inplace=True, subset=['label', 'label_id'])
data.drop_duplicates(keep='first', inplace=True)
data['label_id'] = data['label_id'].astype(int)

In [4]:
data['comment_len'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))

In [5]:
data['comment_len'] = data['comment_len'].apply(lambda len_t: len_t if len_t <= 100 else None)
data = data.dropna(subset=['comment_len'])
data = data.reset_index(drop=True)

In [6]:
normalizer = hazm.Normalizer()
data['normalized_comment'] = data['comment'].apply(lambda c: normalizer.normalize(c.strip()))

In [7]:
data = data[['normalized_comment','label_id']]
data.columns = ['comment', 'label_id']
data.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالی بود همه چه درست و به اندازه و کیفیت خوب، ...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [8]:
MODEL_PATH = 'HooshvareLab/bert-fa-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

In [9]:
input_ids = []
masks = []
for i in data.index:
    encodings = tokenizer.encode_plus(
        data['comment'][i],
        max_length=32,
        truncation=True,
        add_special_tokens=True,
        return_token_type_ids=True,
        return_attention_mask=True,
        padding='max_length',
    )
    input_ids.append(encodings.get("input_ids"))
    masks.append(encodings.get('attention_mask'))


In [10]:
print(len(data), len(input_ids), len(masks))

69156 69156 69156


In [11]:
embeddings = np.multiply(input_ids, masks, dtype=int)
embeddings[0]

array([    2,  5668, 23317,  4244,  2800, 26283,  4875,  8113,  4957,
        2871, 33691,     4,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [17]:
y = (data['label_id'])
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.15)

In [18]:
randomFC = RandomForestClassifier(criterion='gini', n_estimators=1000)

randomFC.fit(X_train, y_train)
y_pred = randomFC.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.60      0.66      5126
           1       0.67      0.79      0.73      5248

    accuracy                           0.70     10374
   macro avg       0.70      0.70      0.69     10374
weighted avg       0.70      0.70      0.69     10374

